
# 02 · Psychometric EDA & Reliability Analysis

This notebook performs:
- Item difficulty & discrimination distributions
- Difficulty vs. discrimination scatter
- Cronbach's alpha (recomputed from `wide_scored`)
- Reliability curve (alpha vs. number of items, ordered by discrimination)


In [ ]:

import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Optional: use your module if available to recompute alpha
try:
    from src.cleaning_psych import cronbach_alpha
except Exception as e:
    cronbach_alpha = None
    print("Note: src.cleaning_psych not found; using local alpha implementation.")

    def cronbach_alpha(X_df):
        X = X_df.to_numpy(dtype=float)
        # Impute column means for NaNs
        col_means = np.nanmean(X, axis=0, keepdims=True)
        inds = np.where(np.isnan(X))
        if inds[0].size:
            X[inds] = np.take(col_means, inds[1])
        k = X.shape[1]
        if k < 2:
            return np.nan
        var_items = X.var(axis=0, ddof=1)
        var_total = X.sum(axis=1).var(ddof=1)
        if var_total == 0:
            return np.nan
        return float((k/(k-1)) * (1 - var_items.sum()/var_total))


In [ ]:

ITEM_STATS_PATH = "output/item_stats.csv"
RESP_SCORES_PATH = "output/respondent_scores.csv"
WIDE_SCORED_PATH = "output/wide_scored.csv"

assert os.path.exists(ITEM_STATS_PATH), f"Missing {ITEM_STATS_PATH}. Re-run the first notebook."
assert os.path.exists(RESP_SCORES_PATH), f"Missing {RESP_SCORES_PATH}. Re-run the first notebook."
assert os.path.exists(WIDE_SCORED_PATH), f"Missing {WIDE_SCORED_PATH}. Re-run the first notebook."

item_stats = pd.read_csv(ITEM_STATS_PATH)
respondent_scores = pd.read_csv(RESP_SCORES_PATH, index_col=0)
wide_scored = pd.read_csv(WIDE_SCORED_PATH, index_col=0)

print(f"Loaded item_stats: {item_stats.shape}")
print(f"Loaded respondent_scores: {respondent_scores.shape}")
print(f"Loaded wide_scored: {wide_scored.shape}")
item_stats.head()


## Difficulty distribution

In [ ]:

plt.figure()
vals = item_stats["difficulty"].dropna().to_numpy()
plt.hist(vals, bins=20)
plt.title("Item Difficulty")
plt.xlabel("Difficulty (mean score)")
plt.ylabel("Count of items")
plt.show()


## Discrimination distribution

In [ ]:

plt.figure()
vals = item_stats["discrimination"].dropna().to_numpy()
plt.hist(vals, bins=20)
plt.title("Item Discrimination (item–total corrected)")
plt.xlabel("Correlation")
plt.ylabel("Count of items")
plt.show()


## Difficulty vs Discrimination (scatter)

In [ ]:

plt.figure()
x = item_stats["difficulty"].to_numpy()
y = item_stats["discrimination"].to_numpy()
plt.scatter(x, y)
plt.title("Difficulty vs Discrimination")
plt.xlabel("Difficulty (mean score)")
plt.ylabel("Discrimination (corr item–total corrected)")
plt.grid(True, linestyle="--", linewidth=0.5, alpha=0.6)
plt.show()


## Cronbach's alpha

In [ ]:

alpha_val = cronbach_alpha(wide_scored)
print(f"Cronbach's alpha (all items): {alpha_val:.3f}")


## Reliability curve (alpha vs number of items)

In [ ]:

# Order items by discrimination descending; remove NaNs first
ordered = item_stats.dropna(subset=["discrimination"]).sort_values("discrimination", ascending=False)
ordered_items = ordered["item_id"].tolist()

alphas = []
ks = []
for k in range(3, len(ordered_items)+1):  # start from 3 items to avoid degenerate alpha
    subset_cols = [c for c in ordered_items[:k] if c in wide_scored.columns]
    if len(subset_cols) < 3:
        continue
    alpha_k = cronbach_alpha(wide_scored[subset_cols])
    ks.append(k)
    alphas.append(alpha_k)

plt.figure()
plt.plot(ks, alphas, marker="o")
plt.title("Reliability Curve: Alpha vs Number of Items (ranked by discrimination)")
plt.xlabel("Number of items")
plt.ylabel("Cronbach's alpha")
plt.grid(True, linestyle="--", linewidth=0.5, alpha=0.6)
plt.show()


## Item quality flags

In [ ]:

# Heuristic flags (adjust thresholds to your scale):
# - Low discrimination < 0.15
# - Extreme difficulty (too easy/hard) outside [0.2, 0.8] for binary or scaled mean bounds for Likert
flags = item_stats.copy()
flags["flag_low_disc"] = flags["discrimination"] < 0.15
flags["flag_easy"] = flags["difficulty"] > 0.8
flags["flag_hard"] = flags["difficulty"] < 0.2
flags["any_flag"] = flags[["flag_low_disc","flag_easy","flag_hard"]].any(axis=1)

flags_sorted = flags.sort_values(["any_flag", "discrimination"], ascending=[False, True])

# Show top 15 flagged
flags_sorted.head(15)


In [ ]:

report_cols = ["item_id","scale_id","difficulty","discrimination","flag_low_disc","flag_easy","flag_hard","any_flag"]
report = flags_sorted[report_cols]
os.makedirs("output", exist_ok=True)
report_path = "output/item_quality_report.csv"
report.to_csv(report_path, index=False)
print(f"Saved: {report_path}")
